In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression

In [2]:
## Dimensões para visualização de datasets
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.8f}'.format)

In [3]:
train = pd.read_csv('train.csv')
train.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.00000000,0.00000000,0.00000000,6/30/2014,7/31/2014,8/31/2014,31.27700000,87.00900000,7.52700000,48.58000000,124.38000000,1.29000000,32.24000000,96.68000000,2.33000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2.23000000,0.00000000,0.28000000,5.29000000,16.04000000,2.33000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,7.53000000,16.04000000,2.61000000,46.34000000,124.38000000,1.01000000,18.75000000,80.61000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,65.09000000,204.99000000,1.01000000,0.00000000,0.00000000,0.00000000,8.20000000,0.63000000,0.00000000,0.38000000,0.00000000,0.00000000,81.21000000,221.68000000,3.63000000,2.43000000,3.68000000,7.79000000,0.83000000,21.08000000,16.91000000,0.00000000,0.00000000,0.00000000,3.26000000,24.76000000,24.71000000,0.00000000,7.61000000,0.21000000,7.46000000,19.96000000,14.96000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,7.46000000,27.58000000,15.18000000,11.84000000,53.04000000,40.56000000,0.00000000,0.00000000,0.66000000,0.00000000,0.00000000,0.00000000,1.11000000,0.69000000,0.00000000,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.00000000,0

In [4]:
train['last_date_of_month_6_n'] =  train['last_date_of_month_6'].apply(lambda x: 1 if x == '6/30/2014' else 0)
train['last_date_of_month_7_n'] =  train['last_date_of_month_7'].apply(lambda x: 1 if x == '7/31/2014' else 0)
train['last_date_of_month_8_n'] =  train['last_date_of_month_8'].apply(lambda x: 1 if x == '8/31/2014' else 0)

In [5]:
train.last_date_of_month_6_n.sum(),train.last_date_of_month_7_n.sum(),train.last_date_of_month_8_n.sum()

(69999, 69600, 69266)

In [6]:
train['last_date_of_month_rep'] = (train.last_date_of_month_6_n + train.last_date_of_month_7_n + train.last_date_of_month_8_n) / 3

In [7]:
train['last_date_of_month_rep'].mean()

0.994609446801621

In [8]:
train['date_of_last_rech_6_day'] = train.date_of_last_rech_6.str.split('/', expand = True)[1].astype(float)
train['date_of_last_rech_7_day'] = train.date_of_last_rech_7.str.split('/', expand = True)[1].astype(float)
train['date_of_last_rech_8_day'] = train.date_of_last_rech_8.str.split('/', expand = True)[1].astype(float)

In [9]:
train['date_of_last_rech_data_6_day'] = train.date_of_last_rech_data_6.str.split('/', expand = True)[1].astype(float)
train['date_of_last_rech_data_7_day'] = train.date_of_last_rech_data_7.str.split('/', expand = True)[1].astype(float)
train['date_of_last_rech_data_8_day'] = train.date_of_last_rech_data_8.str.split('/', expand = True)[1].astype(float)

In [10]:
train['date_of_last_rech_data_6_day'] = train['date_of_last_rech_data_6_day'].astype(float)

In [11]:
feature_names = ["total_rech_data_6",
"max_rech_data_6",
"count_rech_2g_6",
"count_rech_3g_6",
"av_rech_amt_data_6",
"arpu_3g_6",
"arpu_2g_6",
"night_pck_user_6",
"fb_user_6",
"total_rech_data_7",
"max_rech_data_7",
"count_rech_2g_7",
"count_rech_3g_7",
"av_rech_amt_data_7",
"arpu_3g_7",
"arpu_2g_7",
"night_pck_user_7",
"fb_user_7",
"total_rech_data_8",
"max_rech_data_8",
"count_rech_2g_8",
"count_rech_3g_8",
"av_rech_amt_data_8",
"arpu_3g_8",
"arpu_2g_8",
"night_pck_user_8",
"fb_user_8",
"onnet_mou_8",
"offnet_mou_8",
"roam_ic_mou_8",
"roam_og_mou_8",
"loc_og_t2t_mou_8",
"loc_og_t2m_mou_8",
"loc_og_t2f_mou_8",
"loc_og_t2c_mou_8",
"loc_og_mou_8",
"std_og_t2t_mou_8",
"std_og_t2m_mou_8",
"std_og_t2f_mou_8",
"std_og_t2c_mou_8",
"std_og_mou_8",
"isd_og_mou_8",
"spl_og_mou_8",
"og_others_8",
"loc_ic_t2t_mou_8",
"loc_ic_t2m_mou_8",
"loc_ic_t2f_mou_8",
"loc_ic_mou_8",
"std_ic_t2t_mou_8",
"std_ic_t2m_mou_8",
"std_ic_t2f_mou_8",
"std_ic_t2o_mou_8",
"std_ic_mou_8",
"spl_ic_mou_8",
"isd_ic_mou_8",
"ic_others_8",
"onnet_mou_6",
"offnet_mou_6",
"roam_ic_mou_6",
"roam_og_mou_6",
"loc_og_t2t_mou_6",
"loc_og_t2m_mou_6",
"loc_og_t2f_mou_6",
"loc_og_t2c_mou_6",
"loc_og_mou_6",
"std_og_t2t_mou_6",
"std_og_t2m_mou_6",
"std_og_t2f_mou_6",
"std_og_t2c_mou_6",
"std_og_mou_6",
"isd_og_mou_6",
"spl_og_mou_6",
"og_others_6",
"loc_ic_t2t_mou_6",
"loc_ic_t2m_mou_6",
"loc_ic_t2f_mou_6",
"loc_ic_mou_6",
"std_ic_t2t_mou_6",
"std_ic_t2m_mou_6",
"std_ic_t2f_mou_6",
"std_ic_t2o_mou_6",
"std_ic_mou_6",
"spl_ic_mou_6",
"isd_ic_mou_6",
"ic_others_6",
"onnet_mou_7",
"offnet_mou_7",
"roam_ic_mou_7",
"roam_og_mou_7",
"loc_og_t2t_mou_7",
"loc_og_t2m_mou_7",
"loc_og_t2f_mou_7",
"loc_og_t2c_mou_7",
"loc_og_mou_7",
"std_og_t2t_mou_7",
"std_og_t2m_mou_7",
"std_og_t2f_mou_7",
"std_og_t2c_mou_7",
"std_og_mou_7",
"isd_og_mou_7",
"spl_og_mou_7",
"og_others_7",
"loc_ic_t2t_mou_7",
"loc_ic_t2m_mou_7",
"loc_ic_t2f_mou_7",
"loc_ic_mou_7",
"std_ic_t2t_mou_7",
"std_ic_t2m_mou_7",
"std_ic_t2f_mou_7",
"std_ic_t2o_mou_7",
"std_ic_mou_7",
"spl_ic_mou_7",
"isd_ic_mou_7",
"ic_others_7",
"loc_og_t2o_mou",
"std_og_t2o_mou",
"loc_ic_t2o_mou",
"arpu_6",
"arpu_7",
"arpu_8",
"total_og_mou_6",
"total_og_mou_7",
"total_og_mou_8",
"total_ic_mou_6",
"total_ic_mou_7",
"total_ic_mou_8",
"total_rech_num_6",
"total_rech_num_7",
"total_rech_num_8",
"total_rech_amt_6",
"total_rech_amt_7",
"total_rech_amt_8",
"max_rech_amt_6",
"max_rech_amt_7",
"max_rech_amt_8",
"last_day_rch_amt_6",
"last_day_rch_amt_7",
"last_day_rch_amt_8",
"vol_2g_mb_6",
"vol_2g_mb_7",
"vol_2g_mb_8",
"vol_3g_mb_6",
"vol_3g_mb_7",
"vol_3g_mb_8",
"monthly_2g_6",
"monthly_2g_7",
"monthly_2g_8",
"sachet_2g_6",
"sachet_2g_7",
"sachet_2g_8",
"monthly_3g_6",
"monthly_3g_7",
"monthly_3g_8",
"sachet_3g_6",
"sachet_3g_7",
"sachet_3g_8",
"aon",
"aug_vbc_3g",
"jul_vbc_3g",
"jun_vbc_3g",
'last_date_of_month_6_n',
'last_date_of_month_7_n',
'last_date_of_month_8_n',
'date_of_last_rech_6_day',
'date_of_last_rech_7_day',
'date_of_last_rech_8_day',
'date_of_last_rech_data_6_day',
'date_of_last_rech_data_7_day',
'date_of_last_rech_data_8_day'       
]

In [12]:
X_train = train.set_index('id')[feature_names].fillna(0)
X_train.head()

,total_rech_data_6,max_rech_data_6,count_rech_2g_6,count_rech_3g_6,av_rech_amt_data_6,arpu_3g_6,arpu_2g_6,night_pck_user_6,fb_user_6,total_rech_data_7,max_rech_data_7,count_rech_2g_7,count_rech_3g_7,av_rech_amt_data_7,arpu_3g_7,arpu_2g_7,night_pck_user_7,fb_user_7,total_rech_data_8,max_rech_data_8,count_rech_2g_8,count_rech_3g_8,av_rech_amt_data_8,arpu_3g_8,arpu_2g_8,night_pck_user_8,fb_user_8,onnet_mou_8,offnet_mou_8,roam_ic_mou_8,roam_og_mou_8,loc_og_t2t_mou_8,loc_og_t2m_mou_8,loc_og_t2f_mou_8,loc_og_t2c_mou_8,loc_og_mou_8,std_og_t2t_mou_8,std_og_t2m_mou_8,std_og_t2f_mou_8,std_og_t2c_mou_8,std_og_mou_8,isd_og_mou_8,spl_og_mou_8,og_others_8,loc_ic_t2t_mou_8,loc_ic_t2m_mou_8,loc_ic_t2f_mou_8,loc_ic_mou_8,std_ic_t2t_mou_8,std_ic_t2m_mou_8,std_ic_t2f_mou_8,std_ic_t2o_mou_8,std_ic_mou_8,spl_ic_mou_8,isd_ic_mou_8,ic_others_8,onnet_mou_6,offnet_mou_6,roam_ic_mou_6,roam_og_mou_6,loc_og_t2t_mou_6,loc_og_t2m_mou_6,loc_og_t2f_mou_6,loc_og_t2c_mou_6,loc_og_mou_6,std_og_t2t_mou_6,std_og_t2m_mou_6,std_og_t2f_mou_6,std_og_t2c_mou_6,std_og_mou_6,isd_og_mou_6,spl_og_mou_6,og_others_6,loc_ic_t2t_mou_6,loc_ic_t2m_mou_6,loc_ic_t2f_mou_6,loc_ic_mou_6,std_ic_t2t_mou_6,std_ic_t2m_mou_6,std_ic_t2f_mou_6,std_ic_t2o_mou_6,std_ic_mou_6,spl_ic_mou_6,isd_ic_mou_6,ic_others_6,onnet_mou_7,offnet_mou_7,roam_ic_mou_7,roam_og_mou_7,loc_og_t2t_mou_7,loc_og_t2m_mou_7,loc_og_t2f_mou_7,loc_og_t2c_mou_7,loc_og_mou_7,std_og_t2t_mou_7,std_og_t2m_mou_7,std_og_t2f_mou_7,std_og_t2c_mou_7,std_og_mou_7,isd_og_mou_7,spl_og_mou_7,og_others_7,loc_ic_t2t_mou_7,loc_ic_t2m_mou_7,loc_ic_t2f_mou_7,loc_ic_mou_7,std_ic_t2t_mou_7,std_ic_t2m_mou_7,std_ic_t2f_mou_7,std_ic_t2o_mou_7,std_ic_mou_7,spl_ic_mou_7,isd_ic_mou_7,ic_others_7,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,last_date_of_month_6_n,last_date_of_month_7_n,last_date_of_month_8_n,date_of_last_rech_6_day,date_of_last_rech_7_day,date_of_last_rech_8_day,date_of_last_rech_data_6_day,date_of_last_rech_data_7_day,date_of_last_rech_data_8_day
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,1.29000000,2.33000000,0.00000000,0.00000000,0.28000000,2.33000000,0.00000000,0.00000000,2.61000000,1.01000000,0.00000000,0.00000000,0.00000000,1.01000000,0.00000000,0.00000000,0.00000000,7.79000000,16.91000000,0.00000000,24.71000000,0.21000000,14.96000000,0.00000000,0.00000000,15.18000000,0.66000000,0.00000000,0.00000000,48.58000000,32.24000000,0.00000000,0.00000000,2.23000000,5.29000000,0.00000000,0.00000000,7.53000000,46.34000000,18.75000000,0.00000000,0.00000000,65.09000000,0.00000000,8.20000000,0.38000000,2.43000000,0.83000000,0.00000000,3.26000000,0.00000000,7.46000000,0.00000000,0.00000000,7.46000000,0.00000000,0.00000000,1.11000000,124.38000000,96.68000000,0.00000000,0.00000000,0.00000000,16.04000000,0.00000000,0.00000000,16.04000000,124.38000000,80.61000000,0.00000000,0.00000000,204.99000000,0.00000000,0.63000000,0.00000000,3.68000000,21.08000000,0.00000000,24.76000000,7.61000000,19.96000000,0.00000000,0.00000000,27.58000000,0.00000000,0.00000000,0.690000

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(X_train)

In [14]:
y = train['churn_probability']

In [15]:
%%time
clf_LR = LogisticRegression(random_state=12345)
clf_LR.fit(X,y)

Wall time: 3.28 s


c:\users\lincoln.thiago.neves\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=12345)

## Predict test dataset with the first ML model

In [16]:
test = pd.read_csv('test.csv')
test.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,109,0.00000000,0.00000000,0.00000000,6/30/2014,7/31/2014,8/31/2014,91.88200000,65.33000000,64.44500000,31.78000000,20.23000000,23.11000000,60.16000000,32.16000000,34.83000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,24.88000000,20.23000000,21.06000000,18.13000000,10.89000000,8.36000000,0.00000000,13.58000000,0.00000000,0.00000000,0.00000000,0.03000000,43.01000000,44.71000000,29.43000000,6.90000000,0.00000000,2.05000000,42.03000000,7.68000000,26.43000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,48.93000000,7.68000000,28.48000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.03000000,0.00000000,0.00000000,0.00000000,91.94000000,52.39000000,57.94000000,30.33000000,37.56000000,21.98000000,10.21000000,4.59000000,9.53000000,0.26000000,0.00000000,0.00000000,40.81000000,42.16000000,31.51000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,41.73000000,43.56000000,36.26000000,0.54000000,0.34000000,0.39000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,5,5,4,103,90,60,50,30,30,6/21/2014,7/26/2014,8/24/2014,30,30,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1692,0.00000000,0.00000000

In [17]:
test['last_date_of_month_6_n'] =  test['last_date_of_month_6'].apply(lambda x: 1 if x == '6/30/2014' else 0)
test['last_date_of_month_7_n'] =  test['last_date_of_month_7'].apply(lambda x: 1 if x == '7/31/2014' else 0)
test['last_date_of_month_8_n'] =  test['last_date_of_month_8'].apply(lambda x: 1 if x == '8/31/2014' else 0)

In [18]:
test.last_date_of_month_6_n.sum(),test.last_date_of_month_7_n.sum(),test.last_date_of_month_8_n.sum()

(30000, 29798, 29633)

In [19]:
test['last_date_of_month_rep'] = (test.last_date_of_month_6_n + test.last_date_of_month_7_n + test.last_date_of_month_8_n) / 3

In [20]:
test['last_date_of_month_rep'].mean()

0.9936777777777777

In [21]:
test['date_of_last_rech_6_day'] = test.date_of_last_rech_6.str.split('/', expand = True)[1].astype(float)
test['date_of_last_rech_7_day'] = test.date_of_last_rech_7.str.split('/', expand = True)[1].astype(float)
test['date_of_last_rech_8_day'] = test.date_of_last_rech_8.str.split('/', expand = True)[1].astype(float)

In [22]:
test['date_of_last_rech_data_6_day'] = test.date_of_last_rech_data_6.str.split('/', expand = True)[1].astype(float)
test['date_of_last_rech_data_7_day'] = test.date_of_last_rech_data_7.str.split('/', expand = True)[1].astype(float)
test['date_of_last_rech_data_8_day'] = test.date_of_last_rech_data_8.str.split('/', expand = True)[1].astype(float)

In [23]:
test['date_of_last_rech_data_6_day'] = test['date_of_last_rech_data_6_day'].astype(float)

In [24]:
test_ = test.set_index('id')[feature_names].fillna(0)
test_.head()

,total_rech_data_6,max_rech_data_6,count_rech_2g_6,count_rech_3g_6,av_rech_amt_data_6,arpu_3g_6,arpu_2g_6,night_pck_user_6,fb_user_6,total_rech_data_7,max_rech_data_7,count_rech_2g_7,count_rech_3g_7,av_rech_amt_data_7,arpu_3g_7,arpu_2g_7,night_pck_user_7,fb_user_7,total_rech_data_8,max_rech_data_8,count_rech_2g_8,count_rech_3g_8,av_rech_amt_data_8,arpu_3g_8,arpu_2g_8,night_pck_user_8,fb_user_8,onnet_mou_8,offnet_mou_8,roam_ic_mou_8,roam_og_mou_8,loc_og_t2t_mou_8,loc_og_t2m_mou_8,loc_og_t2f_mou_8,loc_og_t2c_mou_8,loc_og_mou_8,std_og_t2t_mou_8,std_og_t2m_mou_8,std_og_t2f_mou_8,std_og_t2c_mou_8,std_og_mou_8,isd_og_mou_8,spl_og_mou_8,og_others_8,loc_ic_t2t_mou_8,loc_ic_t2m_mou_8,loc_ic_t2f_mou_8,loc_ic_mou_8,std_ic_t2t_mou_8,std_ic_t2m_mou_8,std_ic_t2f_mou_8,std_ic_t2o_mou_8,std_ic_mou_8,spl_ic_mou_8,isd_ic_mou_8,ic_others_8,onnet_mou_6,offnet_mou_6,roam_ic_mou_6,roam_og_mou_6,loc_og_t2t_mou_6,loc_og_t2m_mou_6,loc_og_t2f_mou_6,loc_og_t2c_mou_6,loc_og_mou_6,std_og_t2t_mou_6,std_og_t2m_mou_6,std_og_t2f_mou_6,std_og_t2c_mou_6,std_og_mou_6,isd_og_mou_6,spl_og_mou_6,og_others_6,loc_ic_t2t_mou_6,loc_ic_t2m_mou_6,loc_ic_t2f_mou_6,loc_ic_mou_6,std_ic_t2t_mou_6,std_ic_t2m_mou_6,std_ic_t2f_mou_6,std_ic_t2o_mou_6,std_ic_mou_6,spl_ic_mou_6,isd_ic_mou_6,ic_others_6,onnet_mou_7,offnet_mou_7,roam_ic_mou_7,roam_og_mou_7,loc_og_t2t_mou_7,loc_og_t2m_mou_7,loc_og_t2f_mou_7,loc_og_t2c_mou_7,loc_og_mou_7,std_og_t2t_mou_7,std_og_t2m_mou_7,std_og_t2f_mou_7,std_og_t2c_mou_7,std_og_mou_7,isd_og_mou_7,spl_og_mou_7,og_others_7,loc_ic_t2t_mou_7,loc_ic_t2m_mou_7,loc_ic_t2f_mou_7,loc_ic_mou_7,std_ic_t2t_mou_7,std_ic_t2m_mou_7,std_ic_t2f_mou_7,std_ic_t2o_mou_7,std_ic_mou_7,spl_ic_mou_7,isd_ic_mou_7,ic_others_7,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,last_date_of_month_6_n,last_date_of_month_7_n,last_date_of_month_8_n,date_of_last_rech_6_day,date_of_last_rech_7_day,date_of_last_rech_8_day,date_of_last_rech_data_6_day,date_of_last_rech_data_7_day,date_of_last_rech_data_8_day
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69999,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,23.11000000,34.83000000,0.00000000,0.00000000,21.06000000,8.36000000,0.00000000,0.03000000,29.43000000,2.05000000,26.43000000,0.00000000,0.00000000,28.48000000,0.00000000,0.03000000,0.00000000,21.98000000,9.53000000,0.00000000,31.51000000,0.00000000,4.34000000,0.00000000,0.00000000,4.34000000,0.39000000,0.00000000,0.00000000,31.78000000,60.16000000,0.00000000,0.00000000,24.88000000,18.13000000,0.00000000,0.00000000,43.01000000,6.90000000,42.03000000,0.00000000,0.00000000,48.93000000,0.00000000,0.00000000,0.00000000,30.33000000,10.21000000,0.26000000,40.81000000,0.00000000,0.36000000,0.00000000,0.00000000,0.36000000,0.54000000,0.00000000,0.00000000,20.23000000,32.16000000,0.00000000,0.00000000,20.23000000,10.89000000,13.58000000,0.00000000,44.71000000,0.00000000,7.68000000,0.00000000,0.00000000,7.68000000,0.00000000,0.00000000,0.00000000,37.56000000,4.59000000,0.00000000,42.16000000,0.00000000,1.04000000,0.00000000,0.00000000,1.04000000,0.34000000,0.00000000,0

In [25]:
X_test = scaler.fit_transform(test_)

In [26]:
preds_LR = clf_LR.predict(X_test)

In [27]:
preds_LR_ = pd.DataFrame(preds_LR, columns=['churn_probability'])

In [28]:
sub16_LR = pd.concat([test['id'],preds_LR_], axis=1)

In [29]:
sub16_LR.to_csv('sub16_LR.csv', sep=',', index=False)